In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(max_tokens=1024, temperature=0.1)

template = PromptTemplate.from_template("{country_a}와 {country_b} 사이의 거리를 구하세요")
propmt = template.format(country_a="멕시코", country_b="한국")

'멕시코와 한국 사이의 직선 거리는 약 12,500km 입니다.'

In [11]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "넌 최고의 지리학 전문가야. 그리고 넌 {language}로만 대답해야 해."),
        ("ai", "안년, 난 {name}"),
        ("human", "{country_a}와 {country_b} 사이의 거리를 구하세요. 그리고 당신의 이름을 알려주세요.")
    ]
)
prompt = template.format_messages(
    language="English",
    name="Socrates",
    country_a="대한민국",
    country_b="미국"
)
chat.predict_messages(messages=prompt)

AIMessage(content='The distance between South Korea and the United States is approximately 10,000 kilometers. My name is Watermelon Doctor.')